### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/fimochka-sudo/GB_docker_flask_example/blob/main/README.md
2. Далее уже запускать код ниже

In [30]:
import pandas as pd
import urllib.request
import json 
from sklearn.metrics import roc_auc_score,roc_curve,scorer
from urllib import request, parse

def get_prediction(field_array=None, myurl=None):
    """
    Input parameters:
        field_array: python list - contains raw fields needed to make a prediction
        myurl: REST API url
    Returns:
        result: float - model prediction
    """
    result = 0
    age, workclass, education, education_num, marital_status, occupation, relationship, capital_gain, capital_loss, hours_per_week = field_array
    body = {'age':age,
            'workclass':workclass, 
            'education':education, 
            'education_num':education_num, 
            'marital_status':marital_status, 
            'occupation':occupation, 
            'relationship':relationship,
            'capital_gain':capital_gain, 
            'capital_loss':capital_loss, 
            'hours_per_week':hours_per_week, } 

    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    
    return json.loads(response.read())['predictions']

API_URL = "http://0.0.0.0:8180/predict"
# API_URL = "http://127.0.0.1:5000/predict"


# X_test = pd.read_csv("X_test.csv")
# y_test = pd.read_csv("y_test.csv")

In [31]:
features = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
            'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']
target = 'income'

df = pd.read_csv('data/adult.data', sep=',', index_col=False, names=features + [target])
df_test = pd.read_csv('data/adult.test', sep=',', index_col=False, names=features + [target])
# Одна строка в данных "битая", удалим.
df_test=df_test[df_test['age']!='|1x3 Cross validator']

In [32]:
X = df.drop(columns=['income', 'fnlwgt', 'native-country', 'race', 'sex'])
y = df['income']

X_test = df_test.drop(columns=['income', 'fnlwgt', 'native-country', 'race', 'sex'])
y_test = df_test['income']

Посмотрим что за данные у нас есть

In [33]:
X_test.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,capital-gain,capital-loss,hours-per-week
1,25,Private,11th,7.0,Never-married,Machine-op-inspct,Own-child,0.0,0.0,40.0
2,38,Private,HS-grad,9.0,Married-civ-spouse,Farming-fishing,Husband,0.0,0.0,50.0
3,28,Local-gov,Assoc-acdm,12.0,Married-civ-spouse,Protective-serv,Husband,0.0,0.0,40.0


### Getting predictions from REST API

In [48]:

get_prediction(field_array=("25", "Private", "11th", "7.0", "Never-married", "Machine-op-inspct", "Own-child", "0.0", "0.0", "40.0"), myurl=API_URL)

# get_prediction(field_array=("25", "Private", "11th", "7", "Never-married", "Machine-op-inspct", "Own-child", "0", "0", "40"), myurl=API_URL)

'0.0063376245'

In [ ]:
# %%time
# predictions = X_test[['age', 'workclass', 'education', 'education-num', 'marital-status', 'occupation', 'relationship',
#             'capital-gain', 'capital-loss', 'hours-per-week']].iloc[:500].apply(lambda x: get_prediction(x, API_URL), 1)

check performance (roc auc)

In [ ]:
# roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

Давайте подумаем, где наше api может сломаться

In [ ]:
# get_prediction(field_array=("", None, None), myurl=API_URL)

In [ ]:
# get_prediction(field_array=("", "", ""), myurl=API_URL)